# Análise de sentimentos de tweets brasileiros nos períodos anterior e inicial da pandemia de Covid-19

## I - Extração de dados brasileiros e armazenamento em banco de dados

***
### Base de Dados GeoCoV19

O **GeoCoV19** (1) disponibiliza uma base de dados com ***tweets*** que fazem referência à **pandemia da Covid-19**. Esta base contêm mais de **524 milhões de registros** em **62 línguas**, no intervalo de **01/02/2020 a 01/05/2020**. Deste total, **94% possuem informações de localizações**.

Para a inclusão das localizações foram consideradas **informações providas pelo próprio Twitter e localizações extraídas a partir dos textos dos *tweets* (topônimos)**. A extração a partir dos textos considera qualquer menção a algum local como uma informação de localização válida.

Em atendimento aos termos de utilização da *API* do Twitter (2), **os dados disponibilizados não possuem textos de usuários.** O trabalho sugere a utilização de ferramentas para **obtenção do conteúdo integral dos *tweets* através de seus *ids***, procedimento conhecido como *hydrate*.

Os dados coletados tem por objetivo capacitar comunidades de pesquisa a avaliar como as sociedades estão lidando coletivamente com a crise do Covid-19, desenvolver métodos para identificar notícias falsas, entender lacunas de conhecimento, contruir modelos de previsão, desenvolver alertas à doenças, entre outros.

### GeoCoV19 - Dados Brasileiros 

Este trabalho utiliza a base de dados GeoCoV19 para **extrair, selecionar e analisar sentimentos de *tweets* com localizações brasileiras**. Esses *tweets*, correspondentes a um intervalo de 91 dias, compreende, na maioria das cidades, à **períodos anteriores e iniciais da pandemia de Covid-19 no Brasil**, sendo dividido neste trabalho em três períodos: **antes do primeiro caso**, **após o primeiro caso** e **após à primeira morte**, dentro do intervalo da base de dados trabalhada.

Após estes procedimentos, este trabalho apresenta os **resultados das análises de sentimentos** de uma forma consolidada, considerando **cidades que obtiveram as menores e maiores médias de sentimentos**, nos períodos analisados. Busca-se, nestas análises, investigar a **atenção e o comportamento dos usuários do Twitter** em relação à acontecimentos relacionados à evolução da pandemia da Covid-19 no Brasil. 

Este *notebook*, primeiro entre quatro, desenvolvidos neste trabalho, trata do procedimento de **extração de dados brasileiros** da base GeoCov19. Os *notebooks* posteriores tratam, respectivamente, da seleção dos *tweets* extraídos, das análises de sentimentos desses *tweets* e dos resultados obtidos acerca destas análises.

***
### Solução para extração de dados brasileiros

Os dados foram disponibilizados pela base de dados **GeoCoV19** em arquivos diários, no formato *JSON*, compactados no formato *zip*, totalizando **91 arquivos** (3). A solução implementada realiza a **descompactação de cada um desses arquivos, selecionando os registros brasileiros e criando novos arquivos *JSON*** com esses registros.

Os arquivos disponibilizados possuem **informações de localizações dos tweets** (providas pelos atributos *geo*, *place*, *user_location* e *tweet_text*, como detalharemos mais adiante) e também um **atributo informando quais destas localizações foi considerada** (informada pelo atributo *geo_source*). Através deste atributo selecionamos os registros desejados (considerando atributos *geo_source* com a coluna **country_code = "br"**) durante o processamento dos arquivos *JSON*.

Os registros selecionados foram armazenados em um banco de dados **MongoDB**. Em um segundo momento, foram realizados os ***hydrates*** desses *tweets*, utilizando a ferramenta **Twarc**, para que pudessem ser obtidos os **textos dos usuários**. Após a realização deste procedimento, o banco de dados foi atualizado contendo os textos desses *tweets* e outras informações relevantes para as análises realizadas neste trabalho.

### Informações Técnicas

Esta seção detalha os atributos do registro de *tweet* no formato *JSON* disponibilizado pela base de dados GeoCov19. São estes:

1. **tweet_id**: it represents the Twitter provided id of a tweet

2. **created_at**: it represents the Twitter provided "created_at" date and time in UTC 

3. **user_id**: it represents the Twitter provided user id

4. **geo_source**: this field shows one of the four values: (i) coordinates, (ii) place, (iii) user_location, or (iv) tweet_text. The value depends on the availability of these fields. The remaining keys can have the following location_json inside them: {"country_code":"us","state":"California","county":"San Francisco","city":"San Francisco"}.

5. **user_location**: It can have a "location_json" as described above or an empty JSON {}. This field uses the "location" profile meta-data of a Twitter user and represents the user declared location in the text format. We resolve the text to a location. 

6. **geo**: represents the "geo" field provided by Twitter. We resolve the provided latitude and longitude values to locations. It can have a "location_json" as described above or an empty JSON {}.

7. **tweet_locations**: This field can have an array of "location_json" as described above [location_json1, location_json2] or an empty array []. This field uses the tweet content to find toponyms. 

8. **place**: It can have a "location_json" described above or an empty JSON {}. It represents the Twitter-provided "place" field.

A imagem a seguir ilustra um registro de tweet provido pelo GeoCov19, contendo os atributos citados anteriormente.

![title](images/arq_output_json.png)

***
### Importações gerais

In [27]:
import datetime
import pandas as pd

### Configurações iniciais

#### Variáveis utilizadas na extração de dados

Na seção a seguir são definidas variáveis que representam os diretórios utilizados nos procedimentos de extração dos registros de *tweets* do GeoCoV19. Essa estrutura é ilustradas na imagem a seguir.

![title](images/diretorios.png)

Resumidamente, o diretório de ***input*** refere-se a arquivos da base de dados GeoCov19, o diretório de ***output*** refere-se a arquivos gerados durante o processo de extração dos registros e o diretório ***downloads*** refere-se a arquivos contendo registros completos de *tweets* gerados nos procedimentos de *hydrate*.

In [7]:
# Diretório base
#home = '/home/mario/Arquivos'
home = '/media/mario/Dados1/Arquivos'

# Diretório dos arquivos zip 
zip_dir = home + '/input/geo'

# Diretório de descompactação dos arquivos zip (arquivos json)
json_dir = home + '/output/json'

# Diretório dos arquivos json com geolocalizações brasileiras processados a partir dos zips descompactados
geo_dir = home + '/output/geo'

# Diretório dos arquivos de ids
ids_dir = home + '/output/ids'

# Diretório de downloads realizados pelo Twarc
downloads_dir = home + '/downloads'

#### Banco de Dados

Devido ao grande volume de registros de tweets com localizações brasileiras previamente selecionados (mais de 6 milhões, como veremos posteriormente), decidiu-se pelo armazenamento destes registros em um **banco de dados MongoDB** (4).

O MongoDB é um banco de dados *NOSQL* e utiliza **documentos no formato *JSON*** para manipulação de dados, mesmo formato utilizado pelos registros de *tweets* processados neste trabalho. Este fator, além de sua versatilidade e performance (4), foram determinantes para sua escolha.

A seção a seguir realiza a configuração para acesso ao repositório de registros de *tweets*.

In [29]:
# Criando estrutura do banco de dados
from pymongo import MongoClient

# Conexão com o servidor do MongoDB
client = MongoClient('localhost', 27017)

# Conexão com a base de dados do mongoDB
db = client.SpedDB

# Coleção onde serão inseridos os dados
#collection = db.tweets_brasil_test2
collection = db.tweets_brasil

***
### Realização da extração de dados brasileiros

#### Funções auxiliares para extração de dados

In [30]:
import ijson
import json
import os
import zipfile
import numpy as np
import pandas as pd

In [31]:
# Função para extração dos arquivos zip e realização da leitura dos arquivos JSON descompactados
def read_files(zip_dir, json_dir, geo_dir, ids_dir, country_code):
    
    print("Extraindo arquivos...")
    extract_files(zip_dir, json_dir)
    
    json_files = list_files(json_dir, ".json")
        
    total_arquivos = len(json_files)
    total_arquivos_processados = 0
    total_tweets_validos = 0
    
    print(str(total_arquivos) + " arquivo(s) extraídos(s)")
    print("Processando arquivo(s) extraído(s)...")
    
    # Percorrendo arquivos do diretório
    for file in json_files:
        
        total_arquivos_processados = total_arquivos_processados + 1
        
        # Lendo o arquivo JSON extraído     
        num_linhas = sum(1 for line in open(file))
        print("Lendo arquivo '"+get_filename(file)+" com "+str(num_linhas)+" linhas")
        new_tweets = read_tweets(file, country_code)
        print("Tweets válidos encontrados: "+str(len(new_tweets)))
        total_tweets_validos = total_tweets_validos + len(new_tweets)
           
        # Criando dataframe de localização de tweets
        df_geo = create_df_tweets(new_tweets)   
            
        # Escrevendo json com registros de tweets de brasileiros
        filename = get_filename(file)
        csv_path = geo_dir + os.path.sep + filename
        print("-> Gerando arquivo json '"+get_filename(csv_path))
        df_geo.to_json(csv_path, orient='records', force_ascii=False)
        
        # Escrevendo json com registros de ids de tweets brasileiros 
        output_file_path = ids_dir + os.path.sep + filename + '_ids.csv'
        print("-> Gerando arquivo com ids '"+output_file_path)
        df_geo.to_csv(output_file_path, sep=';',encoding='utf-8', index=False, header=False, columns=['tweet_id'])        
            
        df_geo = None
        print("Tweets válidos até o momento: "+str(total_tweets_validos))
                    

In [32]:
# Função para extração dos arquivos zip
def extract_files(zip_dir, json_dir):
    
    zips = list_files(zip_dir, ".zip") 
    
    total_arquivos_processados = 0
    total_arquivos = len(zips)
    
    for file in zips:
        
        total_arquivos_processados = total_arquivos_processados + 1
        
        if (is_file_extracted(file, json_dir)):
            print("-> Arquivo ''"+get_filename(file)+ "' já extraído anteriormente"+" ("+str(total_arquivos_processados)+"/"+str(total_arquivos)+")")
        else:
            # Extraindo arquivo zip
            zip = zipfile.ZipFile(file)
            print("-> Extraindo arquivo '"+get_filename(zip.filename)+"' ("+str(total_arquivos_processados)+"/"+str(total_arquivos)+")")
            zip.extractall(json_dir)
            zip.close()

In [33]:
# Função para a criação de um dataframe a partir dos tweets gerados com os atributos desejados
def create_df_tweets(tweets):
    
    tweet_columns = ['tweet_id','created_at','geo_source','state','city']
    df = pd.DataFrame(tweets, columns = tweet_columns)
    
    # Modificando os tipos de colunas para otimização de espaço
    df.tweet_id = df.tweet_id.astype('int64')
    df.state = df.state.astype('category')
    df.city = df.city.astype('category')
    
    # Informando valores nulos
    df.text = np.nan
    df.score = np.nan
    
    return df

In [20]:
# Função para retornar a localização do tweet a ser considerada 
# (dentre as várias localizações que podem ter sido informadas)
def select_location(tweet, country_code):
    
    if tweet['geo_source'] == 'coordinates':
        return tweet['geo']
    if tweet['geo_source'] == 'place':
        return tweet['place']
    if tweet['geo_source'] == 'user_location':
        return tweet['user_location']
    if tweet['geo_source'] == 'tweet_text':
        for location in tweet['tweet_locations']:
            if location['country_code'] == country_code:
                return location
    else: 
        return {}

In [35]:
# Função para identificar se o tweet que está sendo lido pertence ao país desejado
def is_valid_tweet(tweet, country_code):
    
    # Verificando preliminarmente se os dados pertencem a outros países diferentes do Brasil
    if tweet['geo_source'] == 'geo' and 'country_code' in tweet['geo'] and tweet['geo']['country_code'] != country_code:
        return False
    if tweet['geo_source'] == 'place' and 'country_code' in tweet['place'] and tweet['place']['country_code'] != country_code:
        return False
    if tweet['geo_source'] == 'user_location' and 'country_code' in tweet['user_location'] and tweet['user_location']['country_code'] != country_code:
        return False
    
    # Verificando se as informações de cidades e estados são nulas
    if tweet['geo_source'] == 'geo' and 'country_code' in tweet['geo'] and tweet['geo']['country_code'] == country_code and 'state' in tweet['geo'] and tweet['geo']['state'] != np.nan and 'city' in tweet['geo'] and tweet['geo']['city'] != np.nan:
        return True
    if tweet['geo_source'] == 'place' and 'country_code' in tweet['place'] and tweet['place']['country_code'] == country_code and 'state' in tweet['place'] and tweet['place']['state'] != np.nan and 'city' in tweet['place'] and tweet['place']['city'] != np.nan:
        return True
    if tweet['geo_source'] == 'user_location' and 'country_code' in tweet['user_location'] and tweet['user_location']['country_code'] == country_code and 'state' in tweet['user_location'] and tweet['user_location']['state'] != np.nan  and 'city' in tweet['user_location'] and tweet['user_location']['city'] != np.nan:
        return True
    
    # Caso as informações de localização não estejam presentes nos atributos 'geo', 'place' e 'user_location'
    if tweet['geo_source'] == 'tweet_text':
        for location in tweet['tweet_locations']:
            if location['country_code'] == country_code:
                if 'state' in location and location['state'] != np.nan and 'city' in location and location['city'] != np.nan:
                    return True
                else:
                    return False

    return False  

In [36]:
# Função para a criação de um novo registro de tweet com atributos desejados dos tweets do arquivo original
def create_new_tweet(tweet, country_code):      
    new_tweet = {}    
    location = select_location(tweet, country_code)    
    new_tweet['tweet_id'] = tweet['tweet_id']
    new_tweet['created_at'] = tweet['created_at']
    new_tweet['geo_source'] = tweet['geo_source']
    new_tweet['country_code'] = (location['country_code'] if 'country_code' in location else None)
    new_tweet['state'] = (location['state'] if 'state' in location else None)
    new_tweet['city'] = (location['city'] if 'city' in location else None)   
    return new_tweet

In [5]:
# Função para realizar a leitura de tweets de um país desejado a partir do arquivo JSON
def read_tweets(file, country_code):
    
    with open(file, 'r') as f:
    
        # Array de tweets tratados
        new_tweets = []
        # Realizando leitura iterativa do arquivo (todas as colunas selecionadas)
        objects = ijson.items(f, "", multiple_values=True)
        # Selecionando os tweets desejados
        tweets = (item for item in objects if is_valid_tweet(item, country_code))

        for tweet in tweets:
            new_tweets.append(create_new_tweet(tweet, country_code))
            
    return new_tweets   

In [38]:
def is_file_extracted(file, dir):
    
    filename = get_filename(file)
    filename = filename.replace(".zip",".json")
    filename = dir + os.path.sep + filename
    return os.path.isfile(filename)

In [39]:
def list_files(dir, type):
    
    caminhos = [os.path.join(dir, nome) for nome in os.listdir(dir)]
    arquivos = [arq for arq in caminhos if os.path.isfile(arq)]
    valid_files = [arq for arq in arquivos if arq.lower().endswith(type)]
                
    return sorted(valid_files)

In [40]:
def get_filename(file):
    
    split = file.split(os.path.sep)
    size = len(split)
    filename = split[size-1]
    return filename

#### Realização da leitura dos arquivos zip e seleção de registros brasileiros

Na seção abaixo, é realizada a **seleção dos registros de tweets com localizações brasileiras** a partir dos arquivos providos pela base de dados GeoCoV19. Esta seleção é realizada de acordo com os passos seguintes, ilustradas na imagem a seguir:

**1)** Acesso à base de dados do GeoCov19. A imagem ilustra a página de disponibilização contendo opções de downloads. (3);

**2)** *Downloads* dos 91 arquivos *JSON* compactados no formato *zip* disponibilizados (na versão *all languages*). A imagem ilsutra o diretório contendo arquivos compactados e, em destaque, o conteúdo do arquivo referente ao dia 01/02/2020;

**3)** Descompactação desses arquivos. A imagem ilustra os arquivos *JSON* descompactados e, em destaque, o conteúdo do primeiro registro referente ao dia 01/02/2020, disponibilizado pelo GeoCov19;

**4)** Leitura dos arquivos descompactados e seleção dos registros brasileiros criando novos arquivos *JSON*. A imagem ilustra os arquivos contendo os registros brasileiros selecionados. A imagem ilustra esses arquivos e, em destaque, o conteúdo do primeiro registro com localização brasileira referente ao dia 01/02/2020;

**5)** Geração de arquivos *CSV* contendo os *ids* dos tweets brasileiros para posterior procedimento de *hydrate*. A imagem ilustra os arquivos gerados e, em destaque, o contéudo dos arquivos contendo *ids* dos *tweets* referentes ao dia 01/02/2020.

![title](images/fluxo_extracao.png)

Na seção abaixo, realiza-se a chamada à função de extração exemplificada nas seções anteriores.

In [46]:
# Realizando a extração dos arquivos zip
read_files(zip_dir, json_dir, geo_dir, ids_dir, 'br')

Extraindo arquivos...
-> Extraindo arquivo 'geo_2020-03-31.zip' (1/1)
1 arquivo(s) extraídos(s)
Processando arquivo(s) extraído(s)...
Lendo arquivo 'geo_2020-03-31.json com 9578168 linhas
Tweets válidos encontrados: 94878
-> Gerando arquivo json 'geo_2020-03-31.json
-> Gerando arquivo com ids '/media/mario/Dados2/Arquivos/output/ids/geo_2020-03-31.json_ids.csv
Tweets válidos até o momento: 94878


***
### Armazenamento dos tweets brasileiros no banco de dados

#### Funções auxiliares para armazenamento de dados

In [47]:
# Função para inserção de tweets no banco de dados, a partir de arquivos JSON
def create_db(collection, json_files):
    
    for file in json_files: 
        print('Criando tweets do arquivo '+file)
        with open(file) as json_file: 
            tweets = json.load(json_file)  
            for tweet in tweets:
                date = datetime.datetime.strptime(tweet['created_at'], '%a %b %d %H:%M:%S +0000 %Y')   
                tweet['created_at'] = date
                tweet['period'] = str(date.year) + "_" + str(date.month).zfill(2)
                tweet['text'] = None
                tweet['lang'] = None
                tweet['score'] = None
                collection.insert_one(tweet)

In [48]:
# Função para criação de índice no banco de dados
def create_index(collection, column):
    
    collection.create_index(column)

#### Realização do armazenamento de tweets 

Na seção abaixo, é realizada a **leitura dos arquivos JSON com localizações brasileiras**, contidas no diretório *geo_dir*. Esses arquivos são utilizados como parâmetro de entrada para a função de **criação dos registros de tweets brasileiros no banco de dados MongoDB**.

In [49]:
# Lendo arquivos json com registros brasileiros
json_files = list_files(geo_dir, ".json")

# Criando banco de dados de tweets com as geolocalizações brasileiras
create_db(collection, json_files)

Criando tweets do arquivo /media/mario/Dados2/Arquivos/output/geo/geo_2020-03-31.json


Abaixo, são criados índices no banco de dados para proporcionar maior agilidade nas consultas por atributos.

In [ ]:
# Criando index para outros atributos alvos de selects
create_index(collection, 'tweet_id')
create_index(collection, 'state')
create_index(collection, 'city')
create_index(collection, 'lang')
create_index(collection, 'geo_source')
create_index(collection, 'created_at')

In [50]:
# Retornando quantidade de tweets inseridos
collection.count_documents({})

6281690

In [51]:
# Retornando o primeiro tweet como exemplo
collection.find_one({})

{'_id': ObjectId('5f501410da596ab0c7238d1c'),
 'tweet_id': 1223563091025301507,
 'created_at': datetime.datetime(2020, 2, 1, 11, 5, 48),
 'user_id': '1199299942797500423',
 'geo_source': 'tweet_text',
 'state': 'Rio de Janeiro',
 'city': 'Rio de Janeiro',
 'text': 'Director General of Health Services Dr. Anil Jasinghe confirmed the Chinese woman admitted at IDH has recovered and can be discharged by tomorrow #Lka #SriLanka #coronavirus',
 'score': 0.1027,
 'lang': 'en'}

***
### Hydrate de Tweets

A partir dos **arquivos de *ids* de *tweets*** gerados no procedimento de **leitura dos arquivos *zip***, foram realizados os procedimentos de ***hydrate*** desses registros. Este procedimento consiste no **download, a partir da base de dados do *Twitter*, dos registros completos relativos a um *tweet***. O objetivo deste passo é obter os **textos de usuários** que não puderam ser disponibilizados pela base de dados GeoCoV19, em atendimento aos termos de utilização do *Twitter*.

Este procedimento foi realizado externamente a este *notebook*, via linha de comando, com a utilização da ferramenta ***Twarc*** (4), uma das ferramentas sugeridas pelo artigo da base de dados GeoCov19 para a realização desta tarefa. Para a utilização desta ferramenta é necessária a realização de cadastro específico para utilização da API do *Twitter*, a fim de se obter credenciais para acesso (5).

O *Twarc* retorna a saída do comando dentro de um arquivo *JSON* contendo todos os atributos relativos aos *tweets* retornados a partir dos *ids* informados. A imagem a seguir ilustra uma chamada ao procedimento de *hydrate* com a utilização desta ferramenta.

![title](images/twarc_hydrate_01_02_2020.png)

#### Atualização de textos de tweets no banco de dados

A partir da lista de arquivos retornados, é realizada a **atualização dos registros do banco de dados contendo os textos dos tweets**. Além destes, também realizamos a inserção do **atributo *lang*, referente à língua em que o texto do *tweet* foi escrita**, utilizada posteriormente no refinamento dos tweets selecionados. Este procedimento foi ilustrado no passo 4 da imagem anterior.

In [52]:
# Função para atualização de registro de um tweet no banco de dados
def update_tweets_db(collection, json_files): 
    
    for file in json_files:
        print('Atualizando tweets do arquivo '+file)
        for line in open(file, 'r'):
            tweet = json.loads(line)
            collection.update_one({"tweet_id": tweet['id']}, {'$set':{"text": tweet['full_text'], "lang": tweet['lang']}})

In [55]:
# Listando arquivos json gerados pelo Twarc
twarc_files = list_files(downloads_dir, '.json')

# Atualizando banco de dados com os atributos Text e Lang a partir dos arquivos Json gerados pelo Twarc
update_tweets_db(collection, twarc_files)

Atualizando tweets do arquivo /media/mario/Dados1/Arquivos/downloads/tweets_2020-03-31.json


In [56]:
# Retornando quantidade de tweets com textos não nulos
collection.count_documents({'text': {'$ne':None}})

5104854

In [57]:
# Retornando o primeiro tweet como exemplo
collection.find_one({})

{'_id': ObjectId('5f501410da596ab0c7238d1c'),
 'tweet_id': 1223563091025301507,
 'created_at': datetime.datetime(2020, 2, 1, 11, 5, 48),
 'user_id': '1199299942797500423',
 'geo_source': 'tweet_text',
 'state': 'Rio de Janeiro',
 'city': 'Rio de Janeiro',
 'text': 'Director General of Health Services Dr. Anil Jasinghe confirmed the Chinese woman admitted at IDH has recovered and can be discharged by tomorrow #Lka #SriLanka #coronavirus',
 'score': 0.1027,
 'lang': 'en'}

O procedimento de *hydrate* abordado nesta seção pode ser resumido nestes passos, ilustrados na imagem a seguir.

**1)** Diretório de arquivos contendo *ids* de tweets, pré-requisito para a realização dos procedimentos de  *hydrate*. A imagem ilustra os arquivos *CSV* citados e, em destaque, o conteúdo do arquivo referente ao dia 01/02/2020;

**2)** Para cada arquivo contido no diretório, é executando um comando para realização do *hydrate* com a utilização do *Twarc*. A imagem ilustra o comando realizado para registros referentes ao dia 01/02/2020;

**3)** Cada comando executado gera como saída arquivos *JSON* contendo o conteúdo completo dos *tweets*. A imagem ilustra os arquivos *JSON* gerados, um para cada dia e, em destaque, os atributos referentes ao primeiro tweet do arquivo referente ao dia 01/02/2020;

**4)** Por fim, é realizada a atualização dos registros brasileiros contidos no banco de dados MongoDB inserindo os textos obtidos.

![title](images/fluxo_hydrate.png)

### Resumo

Neste *notebook* foi abordada a solução implementada para a **extração de dados brasileiros** a partir da base de dados do **GeoCov19**. A partir desta extração, foram necessários a realização de procedimentos de ***hydrate***, que consiste em, a partir da base de dados do Twitter, o download do conteúdo completo dos *tweets* desejados. Este procedimento foi necessário pois o **GeoCov19 não disponibiliza textos de usuários** em atendimento aos termos de utilização da *API* do Twitter. Após a obtenção desses textos, estes foram inseridos na base de dados de **registros de tweets brasileiros** no banco de dados MongoDB.

### Fontes

(1) GeoCoV19: A Dataset of Hundreds of Millions ofMultilingual COVID-19 Tweets with Location Information

(2) Twitter API Documentation - https://developer.twitter.com/en/docs

(3) Paper Info, Statics and Downloads - https://crisisnlp.qcri.org/covid19

(4) MongoBD Documentation - https://docs.mongodb.com/manual/reference/command/count/

(5) Pyhton Documentation - https://docs.python.org/3/ 

(6) Twarc - https://github.com/DocNow/twarc

(7) How to Generate API Key, Consumer Token, Access Key for Twitter OAuth - https://themepacific.com/how-to-generate-api-key-consumer-token-access-key-for-twitter-oauth/994/



---

(4) Muhammad Imran, Prasenjit Mitra, Carlos Castillo: Twitter as a Lifeline: Human-annotated Twitter Corpora for NLP of Crisis-related Messages. In Proceedings of the 10th Language Resources and Evaluation Conference (LREC), pp. 1638-1643. May 2016, Portorož, Slovenia.

(6) Iterative JSON parser with a standard Python iterator interface - https://pypi.org/project/ijson/

(7) Make working with large DataFrames easier, at least for your memory - https://towardsdatascience.com/make-working-with-large-dataframes-easier-at-least-for-your-memory-6f52b5f4b5c4

(10) Map Polygon - https://www.keene.edu/campus/maps/tool/

(12) Free Google Translator API for Pyhton - https://pypi.org/project/googletrans/